<a href="https://colab.research.google.com/github/benza613/MS-Apriori-Algorithm/blob/master/InfoGain.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
# set working directory -> you must set the path into which you have uploaded the zipped file
# this is required in the case of colab - not in local mode
%cd /content/drive/My Drive/DMTM

/content/drive/My Drive/DMTM


In [3]:
# list content of drive
%ls

assignments-1-2-3-testData/  IG_Sample1.txt  TestParameter.txt
CL_EVAL_sample1.txt          IG_Sample2.txt  TestTransaction.txt


In [46]:
# READ IG Sample File 
records = []

fileData = open('assignments-1-2-3-testData/assign-2-testData/2-data-1.txt', "r+", encoding='utf-8-sig').readlines()
  
# remove all illegal whitespaces, all newline chars, trailing commas that have no input text after it
records = [line.replace(" ", "").rstrip("\n").rstrip(",") for line in fileData]

records = [list(map(str, line.split(","))) for line in records]

records

[['young', 'FALSE', 'FALSE', 'fair', 'x', 'No'],
 ['young', 'FALSE', 'FALSE', 'good', 'x', 'Mix'],
 ['young', 'TRUE', 'SURE', 'good', 'x', 'Yes'],
 ['young', 'TRUE', 'TRUE', 'fair', 'y', 'Yes'],
 ['young', 'Maybe', 'FALSE', 'fair', 'x', 'No'],
 ['middle', 'FALSE', 'FALSE', 'fair', 'x', 'Mix'],
 ['middle', 'FALSE', 'FALSE', 'good', 'y', 'No'],
 ['middle', 'TRUE', 'TRUE', 'good', 'z', 'Yes'],
 ['middle', 'FALSE', 'SURE', 'excellent', 'y', 'Yes'],
 ['middle', 'FALSE', 'TRUE', 'excellent', 'y', 'Mix'],
 ['old', 'Maybe', 'TRUE', 'excellent', 'y', 'Yes'],
 ['old', 'FALSE', 'TRUE', 'good', 'x', 'Mix'],
 ['old', 'TRUE', 'FALSE', 'good', 'z', 'Yes'],
 ['old', 'TRUE', 'FALSE', 'excellent', 'x', 'Mix'],
 ['old', 'Maybe', 'FALSE', 'fair', 'y', 'No'],
 ['teenager', 'TRUE', 'TRUE', 'good', 'x', 'Mix'],
 ['teenager', 'FALSE', 'SURE', 'good', 'y', 'No'],
 ['teenager', 'TRUE', 'TRUE', 'excellent', 'z', 'Yes'],
 ['teenager', 'Maybe', 'TRUE', 'excellent', 'x', 'Mix'],
 ['teenager', 'TRUE', 'FALSE', 'exce

In [47]:
# get all classes distinct ==? 
# TODO: do we need to upper/lower case them ?
classes = list(set([line[-1] for line in records]))
classes

['No', 'Mix', 'Yes']

In [48]:
import math

def CALC_ENTR(D):
  # Calculate entropy wrt classes 
  var_E_Y = float()

  # loop thru classes
  for idx, c in enumerate(classes):
    n_rows_class = 0

    for r in D:
      # number of rows having this class
      if r[-1] == c :
        n_rows_class += 1  

    # calc class entropy here 
    pr_c = n_rows_class / len(D)
    var_E_Y += ( -pr_c) * math.log(pr_c,2)

    print(c, n_rows_class) 
  return var_E_Y
  
E_Y = CALC_ENTR(records)

No 5
Mix 7
Yes 8


In [49]:
import math

def fn_attr_entropy(D_i):
  # Calculate entropy wrt classes 
  E_x = float()

  # loop thru classes
  for idx, c in enumerate(classes):
    n_rows_class = 0

    for r in D_i:
      # number of rows having this class
      if r[-1] == c :
        n_rows_class += 1  

    if n_rows_class == 0:
      continue

    # calc class entropy here 
    pr_c = n_rows_class / len(D_i)
    E_x += ( -pr_c) *  math.log((pr_c),2)

    #print(c, n_rows_class) 
  return E_x


In [50]:
def IG_ATTR_LOOPER(IDX, precision_req):
  # Calculate a dict of attribute value counts for records[i][IDX]
  attr_vals = {}

  for r in records:
    if r[IDX] in attr_vals.keys():
      attr_vals[r[IDX]] += 1
    else:
      attr_vals[r[IDX]] = 1

  # partial entropies 
  E_vi = float()

  #print(attr_vals)

  for val in attr_vals.keys():
    D_i = [r for r in records if r[IDX] == val]
    pr_c = attr_vals[val] / len(records)

    # calculate sum of partial entropies here 
    E_vi += pr_c * fn_attr_entropy(D_i)


  print('\nENTR for '+str(IDX)+' : ', format(E_vi, precision_req))
  #print('GAIN for '+str(IDX)+' : ', format(E_Y - E_vi, precision_req))
  return '(IG '+ format(E_Y - E_vi, precision_req) + ')'



In [54]:
# SPECIFY ATTRIBUTE COLUMN YOU WANT HERE 
precision_req =  '.3f'

print('TOTAL ENTR : ', E_Y)
for IDX in range(len(records[0])-1):
   print(IG_ATTR_LOOPER(IDX, precision_req))

TOTAL ENTR :  1.5588718484453605

ENTR for 0 :  1.522
(IG 0.037)

ENTR for 1 :  1.187
(IG 0.372)

ENTR for 2 :  1.226
(IG 0.332)

ENTR for 3 :  1.312
(IG 0.247)

ENTR for 4 :  1.058
(IG 0.501)


In [56]:
STUDENT_ID = 'STUDENT_ID'

result =  '(' + STUDENT_ID + '\n' 
result += IG_ATTR_LOOPER(0, precision_req)
result += '\n)'
print(result)

f = open('assignments-results/assign2/'+STUDENT_ID+'_2.txt', "w")
f.write(result)
f.close()


ENTR for 0 :  1.522
(STUDENT_ID
(IG 0.037)
)
